In [12]:
import os
import requests
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.windows import Window

# Function to download SRTM data for a given latitude and longitude tile
def download_srtm_tile(lat, lon, username, password, output_dir="srtm_data"):
    session = requests.Session()
    session.auth = (username, password)
    
    lat_str = f"N{abs(lat):02d}" if lat >= 0 else f"S{abs(lat):02d}"
    lon_str = f"E{abs(lon):03d}" if lon >= 0 else f"W{abs(lon):03d}"
    tile_name = f"{lat_str}{lon_str}.hgt.zip"
    
    base_url = f"https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip"
    print(base_url)
    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, tile_name)

    response = session.get(base_url)
    if response.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {tile_name}")
    else:
        print(f"Failed to download {tile_name}. Status code: {response.status_code}")
        return None

    return file_path

# Function to calculate slope
def calculate_slope(elevation, transform):
    x, y = np.gradient(elevation, transform.a, transform.e)
    slope = np.sqrt(x**2 + y**2)
    slope_degrees = np.arctan(slope) * 180 / np.pi
    return slope_degrees

# Function to calculate average slope over 500x500 meter grids
def average_slope_500m(slope_array, transform, grid_size_m=500):
    pixel_size = transform.a  # Assuming square pixels, this is the pixel size in meters
    grid_size_px = int(grid_size_m / pixel_size)
    
    # Reshape the slope array into 500x500 meter grid blocks
    slope_blocks = slope_array.reshape(
        slope_array.shape[0] // grid_size_px,
        grid_size_px,
        slope_array.shape[1] // grid_size_px,
        grid_size_px
    )
    
    # Calculate the average slope for each grid block
    avg_slope = slope_blocks.mean(axis=(1, 3))
    
    return avg_slope

# Download and process all tiles for California
def process_california_slope(username, password):
    lat_range = range(32, 43)  # From 32°N to 42°N
    lon_range = range(-124, -113)  # From 114°W to 124°W (negative for Western Hemisphere)
    
    tiles = []
    
    # Download and store tiles
    for lat in lat_range:
        for lon in lon_range:
            file_path = download_srtm_tile(lat, lon, username, password)
            if file_path:
                tiles.append(file_path)
    
    # Open all downloaded tiles and merge them into one raster
    datasets = [rasterio.open(tile) for tile in tiles]
    mosaic, mosaic_transform = merge(datasets)
    
    # Calculate the slope from the merged elevation data
    slope_array = calculate_slope(mosaic[0], mosaic_transform)
    
    # Calculate average slope in 500x500 meter grids
    avg_slope = average_slope_500m(slope_array, mosaic_transform)
    
    # Cleanup
    for ds in datasets:
        ds.close()
    
    return avg_slope

# Example usage
username = "mashhafif"
password = "ToshG3091010!"

# Calculate average slope for California
avg_slope_california = process_california_slope(username, password)

# Display the result (this is just for demonstration, you can save it to a file or process further)
print("Average slope (in degrees) over 500m x 500m grids for California:")
print(avg_slope_california)


https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip
Failed to download N32W124.hgt.zip. Status code: 401
https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip
Failed to download N32W123.hgt.zip. Status code: 401
https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip
Failed to download N32W122.hgt.zip. Status code: 401
https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip
Failed to download N32W121.hgt.zip. Status code: 401
https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip
Failed to download N32W120.hgt.zip. Status code: 401
https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip


SSLError: HTTPSConnectionPool(host='e4ftl01.cr.usgs.gov', port=443): Max retries exceeded with url: /DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:997)')))

In [7]:
import ee
import geemap.core as geemap

In [13]:
import requests

url = "https://e4ftl01.cr.usgs.gov/MOLA/SRTMGL1.003/2000.02.11/N32W124.hgt.zip"
output_path = "N32W124.hgt.zip"

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded {output_path}")
else:
    print(f"Failed to download {url}. Status code: {response.status_code}")

Failed to download https://e4ftl01.cr.usgs.gov/MOLA/SRTMGL1.003/2000.02.11/N32W124.hgt.zip. Status code: 404


In [18]:
import requests
from requests.auth import HTTPBasicAuth
import ssl
from urllib3 import poolmanager

class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        context.minimum_version = ssl.TLSVersion.TLSv1_2
        kwargs['ssl_context'] = context
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

# Correct URL for the file you want to download
url = "https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip"

# Earthdata Login credentials
username = "mashhafif"
password = "ToshG3091010!"

# Output path for the downloaded file
output_path = "N32W116.SRTMGL3.hgt.zip"

session = requests.Session()
session.mount('https://', TLSAdapter())

# Make the request with HTTP Basic Auth
response = session.get(url, auth=HTTPBasicAuth(username, password), stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to a file
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded {output_path}")
else:
    print(f"Failed to download {url}. Status code: {response.status_code}")


Failed to download https://e4ftl01.cr.usgs.gov/DP109/SRTM/SRTMGL3.003/2000.02.11/N32W116.SRTMGL3.hgt.zip. Status code: 401
